# Julia for FEM: ``Gridap.jl``
Gridap provides a set of tools for the grid-based approximation of partial differential equations (PDEs) written in the Julia programming language. The library currently supports linear and nonlinear PDE systems for scalar and vector fields, single and multi-field problems, conforming and nonconforming finite element (FE) discretizations, on structured and unstructured meshes of simplices and n-cubes. Gridap is extensible and modular. One can implement new FE spaces, new reference elements, use external mesh generators, linear solvers, post-processing tools, etc.

More information is available at https://juliapackages.com/p/gridap.

## Installation
Gridap is a registered package in the official Julia package registry. Thus, the installation of Gridap is straight forward using the Julia's package manager. Open the Julia REPL, type ``]`` to enter package mode, and install as follows
```
pkg> add Gridap
```

## Integration with ``Gmsh``
We can directly use the geometries defined using ``Gmsh`` in ``Gridap.jl`` using the [GridapGmsh](https://github.com/gridap/GridapGmsh.jl) plugin. The model can be loaded directly with
```julia
using Gridap, GridapGmsh

model = GmshDiscreteModel("path/to/model.msh")
```
Named physical groups (e.g. domains or boundaries) can be accessed in ``Gridap`` to assign properties, such as material properties and boundary conditions. An example of this is given below.

## Visualization
Visualization is very easy using ``Gridap``. There are two main possibilities:
- Using an external program for viewing ``.vtk`` files such as [ParaView](https://www.paraview.org/)
- Using the [GridapMakie.jl](https://github.com/gridap/GridapMakie.jl) plugin for plotting with [Makie.jl](https://github.com/JuliaPlots/Makie.jl).

## Tutorials
The github repository also has a set of tutorials for working with ``Gridap.jl``. This is mostly focussed on mechanical and fluid dynamic problems, but can be adapted for our power engineering purposes.
- https://gridap.github.io/Tutorials/stable/
- https://gridap.github.io/Tutorials/dev/

# Example: High-Voltage Cable

In [ ]:
using gmsh

using Gridap, GridapGmsh

## Define Geometry

In [ ]:
R_cond = 19.1e-3;
R_ins  = 18.4e-3;
R_sh   = 1e-3;
R_jac  = 8e-3;

r_cond = R_cond;             # Conductor
r_ins  = r_cond + R_cond;    # Insulator
r_sh   = r_ins + R_sh;       # Sheath
r_jac  = r_sh + R_jac;       # Jacket

V = 245e3 * sqrt(2 / 3); # Operating voltage

# Mesh density
mshd_cond = R_cond / 10; 
mshd_ins  = R_ins / 10;
mshd_sh   = R_sh / 5;
mshd_jac  = R_jac / 2;

In [ ]:
gmsh.finalize()
gmsh.initialize()
gmsh.option.setNumber("General.Terminal", 1)
gmsh.model.add("gridap_example")

geo = gmsh.model.geo;

## Points
geo.addPoint(0, 0, 0, 1, 1)
geo.addPoint(+r_cond, 0, 0, mshd_cond, 2)
geo.addPoint(-r_cond, 0, 0, mshd_cond, 3)
geo.addPoint(+r_ins, 0, 0, mshd_sh, 4)
geo.addPoint(-r_ins, 0, 0, mshd_sh, 5)
geo.addPoint(+r_sh, 0, 0, mshd_sh, 6)
geo.addPoint(-r_sh, 0, 0, mshd_sh, 7)
geo.addPoint(+r_jac, 0, 0, mshd_jac, 8)
geo.addPoint(-r_jac, 0, 0, mshd_jac, 9)

## Curves
geo.addCircleArc(2, 1, 3, 1)
geo.addCircleArc(3, 1, 2, 2)
geo.addCircleArc(4, 1, 5, 3)
geo.addCircleArc(5, 1, 4, 4)
geo.addCircleArc(6, 1, 7, 5)
geo.addCircleArc(7, 1, 6, 6)
geo.addCircleArc(8, 1, 9, 7)
geo.addCircleArc(9, 1, 8, 8)

## Surfaces
geo.addCurveLoop([1, 2], 1)
geo.addCurveLoop([3, 4], 2)
geo.addCurveLoop([5, 6], 3)
geo.addCurveLoop([7, 8], 4)

geo.addPlaneSurface([1], 1)
geo.addPlaneSurface([2, 1], 2)
geo.addPlaneSurface([3, 1, 2], 3)
geo.addPlaneSurface([4, 1, 2, 3], 4)

## Define domains
geo.addPhysicalGroup(2, [1], 1) # Conductor
geo.addPhysicalGroup(2, [2], 2) # Dielectric
geo.addPhysicalGroup(2, [3], 3) # Sheath
geo.addPhysicalGroup(2, [4], 4) # Jacket
gmsh.model.setPhysicalName(2, 1, "Dielectric")
gmsh.model.setPhysicalName(2, 2, "Conductor")
gmsh.model.setPhysicalName(2, 3, "Sheath")
gmsh.model.setPhysicalName(2, 4, "Jacket")

geo.addPhysicalGroup(1, [1, 2], 1) # Conductor boundary
geo.addPhysicalGroup(1, [3, 4], 2) # Sheath boundary
geo.addPhysicalGroup(0, [2, 3], 1) # Conductor nodes
geo.addPhysicalGroup(0, [4, 5], 2) # Sheath nodes

gmsh.model.setPhysicalName(1, 1, "D1")
gmsh.model.setPhysicalName(1, 2, "D2")
gmsh.model.setPhysicalName(0, 1, "D1p")
gmsh.model.setPhysicalName(0, 2, "D2p")

# Generate mesh and save
gmsh.model.geo.synchronize()
gmsh.model.mesh.generate(2)

gmsh.write("geo/gridap_example.msh")

## Gridap

In [ ]:
# Load geometry from gmsh
model = GmshDiscreteModel("geo/gridap_example.msh")

# Create a reference finite element space to use for the construction of test space V0
order = 2;
reffe = ReferenceFE(lagrangian, Float64, order); # 2 = quadratic elements (P2)

# The test space is created from the reference FE with
#  conformity = H1: shape functions are continuous
#  D1 and D2 being the boundary points with Dirichlet condition applied
V0 = TestFESpace(model, reffe; conformity = :H1, dirichlet_tags = ["D1", "D1p", "D2", "D2p"]);

# Trial space is constructed from the test space
#  the array indicates the values of the Dirichlet condition at the ``dirichlet_tags``
Ug = TrialFESpace(V0, [V, V, 0, 0]);

# Triangulation of the domain
#  degree must be at least 2 times the order of the reference space
degree = 2 * order;
Ω = Triangulation(model);
dΩ = Measure(Ω, degree);

In [ ]:
# Construct weak form
lhs(u,v) = ∫( ∇(v) ⋅ ∇(u) )dΩ;   # Bilinear term
rhs(v)   = 0;                    # Linear term

# Construct FE operator with LHS, RHS, and test and trial spaces
#  This makes an affine mapping to the reference element
op = AffineFEOperator(lhs, rhs, Ug, V0);

# Solve the linear FE system with LU solver
ls = LUSolver()
solver = LinearFESolver(ls)
uh = solve(solver, op);

# Post-process for the electric field
E = ∇(uh);

## Visualization

In [ ]:
using CairoMakie
using GridapMakie

In [ ]:
fig, ax , plt = plot(Ω, uh * 1e-3, colormap=:bluesreds)
Colorbar(fig[1,2], plt, label = "Electric Potential [kV]")
ax.aspect = AxisAspect(1)
save("images/gridap_ex_V.png", fig)
current_figure()

![Result: Electric Potential](images/gridap_ex_V.png)

In [ ]:
fig, ax , plt = plot(Ω, E * 1e-6, colormap=:bluesreds)
Colorbar(fig[1,2], plt, label = "Electric Field Strength [kV/mm]")
ax.aspect = AxisAspect(1)
save("images/gridap_ex_E.png", fig)
current_figure()

![Result: Electric Field Strength](images/gridap_ex_E.png)